# Simulations-App

In [2]:
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html, Input, Output, State
import base64
import plotly.graph_objects as go
import os
import pandas as pd
import calendar
import plotly.express as px

df = None
data = None

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, 'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css'])

##############################################################################################################
# Layout
##############################################################################################################
def build_layout():

    accordion_item = dbc.AccordionItem(
        children=readme_blocks(),
        title="Smart Grid Strategist",
    )
    
    return dbc.Container(
        fluid=True,
        children=[
            dbc.Row(
                dbc.Col(
                    dbc.Accordion(
                        [
                            accordion_item
                        ],
                        start_collapsed=True,
                    ),
                    style={'margin-bottom': '10px'}
                )
            ),
            dbc.Row(
                dbc.Col(
                    dbc.Accordion(
                        [
                            dbc.AccordionItem(
                                [
                                    dbc.Row(
                                        [
                                            dbc.Col(
                                                [
                                                    html.Label("Kategorie des Letztverbrauchers:", style={'margin-bottom': '10px'}),                                                    
                                                    dbc.RadioItems(id='user-type-radio',
                                                                   options=[
                                                                       {'label': 'Consumer', 'value': 'consumer'},
                                                                       {'label': 'Prosumer', 'value': 'prosumer'},
                                                                       {'label': 'Flexumer', 'value': 'flexumer'}
                                                                   ],
                                                                   value='consumer',
                                                                   inline=True,
                                                                   style={'margin-bottom': '10px'}
                                                                  ),
                                                    html.Img(id='dynamic-image', style={'width': '100%', 'height': 'auto', 'margin-bottom': '10px'}),
                                                          
                                                ],
                                                width=6
                                            ),
                                            dbc.Col(
                                                [
                                                    html.Label("Jährlicher Stromverbrauch (kWh):"),
                                                    dbc.Input(id='annual-consumption-input', type='number', step=1, value=2500, style={'margin-bottom': '10px'}),
                                                    html.Label("Aktueller Stromtarif (Ct./kWh):"),
                                                    dbc.Input(id='current-tariff-input', type='number', step=0.01, value=34.37, style={'margin-bottom': '10px'}),
                                                    html.Label("Grundpreis (€/Monat):"),
                                                    dbc.Input(id='current-base-input', type='number', step=0.01, value=14.96, style={'margin-bottom': '10px'}),
                                                    html.Label("Stromerzeugungskapazität der PV-Anlage:", style={'margin-bottom': '10px'}),
                                                    dbc.RadioItems(id='pv-capacity', inline=True, style={'margin-bottom': '10px'}),
                                                    html.Label("Stromspeicherkapazität:", style={'margin-bottom': '10px'}),
                                                    dbc.RadioItems(id='battery-capacity', inline=True, style={'margin-bottom': '10px'})                                    
                                                ],
                                                width=6
                                            )
                                        ]
                                    ),
                                    dbc.Row([dbc.Col(dbc.Button('Simulieren', id='simulate-button', n_clicks=0, color="primary", className="mt-3", style={'width': '100%'}))])
                                ],
                                 title="Simulator"
                            )
                        ]
                    ),
                    style={'margin-bottom': '10px'}
                )
            ),
            dbc.Row(
                dbc.Col(
                    dbc.Accordion(
                        [
                            dbc.AccordionItem(
                                [
                                    dbc.Row(
                                        [
                                            dbc.Col(
                                                dcc.DatePickerRange(display_format='DD/MM/YYYY'),  # TODO
                                                width=4
                                            ),
                                            dbc.Col(
                                                dbc.Button('Simulationsbericht generieren', id='generate-report-button', color='secondary'),  # TODO
                                                width=4
                                            ),
                                            dbc.Col(
                                                dbc.Button(id='save-button',
                                                           children=[
                                                                html.I(className="fas fa-download mr-2"),
                                                               'Simulationsdaten herunterladen'
                                                           ],
                                                           color='secondary',
                                                          ),
                                                width=4
                                            )
                                        ]
                                    ),
                                    dbc.Row(
                                        dbc.Col(
                                            dbc.CardBody(
                                                dbc.Spinner(html.Div(id='simulation-results'))
                                            )
                                        )
                                    )
                                ],
                                title="Analyse"
                            )
                        ],
                        id='simulation-view',
                        style={'display': 'none'},
                        
                    )
                )
            )
        ],
         style={"background-color": "#333", "height": "100%"}
    )

##############################################################################################################
# Methods
##############################################################################################################
def readme_blocks():
    return [
        html.P("Willkommen bei Smart Grid Strategist!",style = {'margin': '2', 'font-weight': 'bold'}),
        html.P("SGS (Smart Grid Strategist) ist eine Anwendung, die im Rahmen der Bachelorarbeit mit dem Titel 'Effizientes Energiemanagement in deutschen Haushalten: Potenzialanalyse durch Integration von Smart-Meter-Daten und maschinellem Lernen im Kontext dynamischer Tarifmodelle' entwickelt wurde. Diese Anwendung bietet einen umfassenden Überblick über den Stromverbrauch in Haushalten und ermöglicht es den Benutzern, ihr individuelles Stromverbrauchsverhalten zu verstehen und Einspar- und Optimierungsmöglichkeiten zu identifizieren.",style = {'margin': '2'}),
        html.P("Eigenschaften der Anwendung",style = {'margin': '2', 'font-weight': 'bold'}),
        html.P("Die Anwendung besteht aus drei Hauptkomponenten:",style = {'margin': '2'}),
          html.Ul([
            html.Li("Simulator: Die Anwendung verfügt über einen Simulator, der drei Kategorien von Letztverbrauchern umfasst: Consumer, Prosumer und Flexumer. Jede Kategorie hat spezifische Merkmale, die es ermöglichen, ihr Verhalten in verschiedenen Szenarien zu simulieren."),
            html.Li("Analysegrafiken: Die Ergebnisse der Simulationen werden in klaren und visuellen Grafiken präsentiert, um die Analyse der komplexen Beziehungen zwischen den Variablen zu erleichtern."),
            html.Li("Machine Learning-Vorhersagen: Es werden maschinelles Lernen verwendet, um das Verhalten der simulierten Variablen vorherzusagen und mögliche Trends für das laufende Jahr aufzuzeigen.")
        ]),    
        html.P("Kategorien von Letztverbrauchern",style = {'margin': '2', 'font-weight': 'bold'}),
        html.Ul([
            html.Li("Consumer: traditionelle Stromverbraucher, die Strom aus dem Netz beziehen."),
            html.Li("Prosumer: Stromverbraucher, die gleichzeitig Strom erzeugen und verbrauchen. In den Forschungsszenarien ist diese Kategorie mit Photovoltaikanlagen und ohne Batterien ausgestattet, d. h. es gibt keine Möglichkeit, Strom zu speichern."),
            html.Li("Flexumer: Stromverbraucher, die gleichzeitig Strom erzeugen und verbrauchen. In den Forschungsszenarien ist diese Kategorie mit PV-Anlagen und Batterien ausgestattet, d. h. es besteht die Möglichkeit, Strom zu speichern. Darüber hinaus ist diese Kategorie mit zwei Stromverbrauchskomponenten ausgestattet, die durch ein Überwachungssystem gesteuert werden können.")
        ]),
        html.P("Funktionsweise der Simulationen",style = {'margin': '2', 'font-weight': 'bold'}),
        html.Ul([
            html.Li("Der Zeitraum für die in den Simulationen eingesetzten Parameter entspricht dem Jahr 2023."),
            html.Li("Die Simulation erstellt ein synthetisches Standardlastprofil (SLP) anhand der vorgegebenen Parameter. Sie berechnet auch die tatsächlichen Stromkosten in 15-Minuten-Intervallen des festen Stromtarifs und simuliert die potenziellen Kosten eines dynamischen Stromtarifs. Die Berechnung der potenziellen Kosten eines dynamischen Stromtarifs basiert auf dem Strompreis auf dem Day-Ahead-Markt.", style = {'margin': '0'}),
            html.Li("Bei Szenarien mit PV-Anlagen wird ein synthetisches Stromerzeugungsprofil für das Jahr 2023 erstellt, basierend auf der Leistung der PV- Anlage.",style = {'margin': '0'}),
            html.Li("Bei Szenarien mit Batterien und steuerbaren Komponenten wird das Management dieser Komponenten im Hinblick auf den Stromverbrauch und die Stromerzeugung während des gesamten Jahres simuliert."),
            html.Li("Die Anwendung nutzt eine Komponente für maschinelles Lernen, um Vorhersagen über das Verhalten der simulierten Variablen zu erstellen und zeigt die mögliche Entwicklung dieser Variablen für das Jahr 2024."),
        ]),
        html.P("Grafiken", style = {'margin': '2', 'font-weight': 'bold'}),
        html.P("Die Simulationsergebnisse werden in anschaulichen Grafiken präsentiert, um eine einfache Analyse der komplexen Zusammenhänge zu ermöglichen. Diese Grafiken bieten einen klaren Überblick über die Leistung der verschiedenen Szenarien, erleichtern den Vergleich und die Identifizierung von wichtigen Mustern und Trends."),
        
    ]

def load_image(user_type):
    base_dir = 'Bilder/'
    if user_type == 'consumer':
        image_filename = 'Consumer.png'
    elif user_type == 'prosumer':
        image_filename = 'Prosumer.png'
    elif user_type == 'flexumer':
        image_filename = 'Flexumer.png'
    else:
        image_filename = 'Consumer.png'

    with open(base_dir + image_filename, 'rb') as f:
        image = f.read()
    encoded_image = base64.b64encode(image).decode()

    return f"data:image/png;base64,{encoded_image}"

def load_data():
    global data, df
    excel_file_path = 'Output-Data/ETL-Prozess_Ergebnisse.xlsx'
    
    if os.path.exists(excel_file_path):
        df = pd.read_excel(excel_file_path, usecols=['Datum', 'SLP-Basisprozentwert', 'PV-10kWp','PV-20kWp','PV-30kWp', 'Strommarktpreis [€/kWh]'], nrows=35040)
        data = df[['Datum', 'SLP-Basisprozentwert']].copy()
    else:
        print(f"Error: No se pudo encontrar el archivo {excel_file_path}")

def simulate_power_consumption(annual_consumption, tariff_input):
    global data
    try:
        annual_consumption = float(annual_consumption)
        power_consumption = data['SLP-Basisprozentwert'] * annual_consumption / 100
        data['Stromverbrauch [kWh]'] = power_consumption
        return True
    except Exception as e:
        print(f"Fehler bei der Simulation des Stromverbrauchs: {e}")
        return False
    
def simulate_power_production(pv_capacity):
    global data
    try:  
        if pv_capacity == 'PV-1kWp':
            power_production = df['PV-10kWp']/10
            data['Stromerzeugung [kWh]'] = power_production
        if pv_capacity == 'PV-5kWp':
            power_production = df['PV-10kWp']/2
            data['Stromerzeugung [kWh]'] = power_production
        if pv_capacity == 'PV-10kWp':
            power_production = df['PV-10kWp']
            data['Stromerzeugung [kWh]'] = power_production
        if pv_capacity == 'PV-15kWp':
            power_production = (df['PV-10kWp']/2 + tmp['PV-10kWp'])
            data['Stromerzeugung [kWh]'] = power_production
        if pv_capacity == 'PV-20kWp':
            power_production = df['PV-10kWp']*2
            data['Stromerzeugung [kWh]'] = power_production
        if pv_capacity == 'PV-25kWp':
            power_production = (df['PV-10kWp']*2 + tmp['PV-10kWp']/2)
            data['Stromerzeugung [kWh]'] = power_production
        if pv_capacity == 'PV-30kWp':
            power_production = df['PV-10kWp']*3
            data['Stromerzeugung [kWh]'] = power_production
        
        return True
    except Exception as e:
        print(f"Fehler bei der Simulation der Stromerzeugung: {e}")
        return False   
    

def simulate_power_price(annual_consumption, current_tariff, current_base_price):
    global data, df 
    annual_consumption = float(annual_consumption)
    current_tariff = float(current_tariff)
    current_base_price = float(current_base_price)
        
    fixed_tariff_values = (data['Stromverbrauch [kWh]'] * (current_tariff/100)) + ((current_base_price * 12) / 35040)
    variable_tariff_values = (data['Stromverbrauch [kWh]'] * df['Strommarktpreis [€/kWh]']) + ((current_base_price * 12) / 35040)
        
    data['Fixer Stromtarif [€/kWh]'] = fixed_tariff_values
    data['Dynamischer Stromtarif [€/kWh]'] = variable_tariff_values
    data['Strommarktpreis [€/kWh]'] = df['Strommarktpreis [€/kWh]']
    return True

def simulate_battery_usage(battery_capacity):
    global data
    try:
        batt_capacity = 0.0
        if battery_capacity == '2,5 kWh':
             batt_capacity = 2.5
        if battery_capacity == '5,1 kWh':
             batt_capacity = 5.1
        if battery_capacity == '7,7 kWh':
             batt_capacity = 7.7
        if battery_capacity == '10,2 kWh':
             batt_capacity = 10.2
        if battery_capacity == '12,8 kWh':
             batt_capacity = 12.8
            
        battery_capacity = float(batt_capacity)

        data['Stromspeicher [kWh]'] = 0.0
        
        batt_15mins_peak = batt_capacity / 4
     
        for index, row in data.iterrows():
            consumption = row['Stromverbrauch [kWh]']
            production = row['Stromerzeugung [kWh]']
            
            if production == 0:
                batt_consumption = consumption if consumption <= batt_15mins_peak else batt_15mins_peak
                if batt_consumption <= battery_capacity:
                    data.at[index, 'Stromspeicher [kWh]'] = battery_capacity - batt_consumption
                    battery_capacity -= batt_consumption
                else:
                    data.at[index, 'Stromspeicher [kWh]'] = 0.0
                    battery_capacity = 0.0
            else:
                if (production - consumption) > 0:
                    production_rest = batt_15mins_peak if (production - consumption) >= batt_15mins_peak else (production - consumption)
                    if (data.at[index - 1, 'Stromspeicher [kWh]'] + production_rest) <= batt_capacity:
                        data.at[index, 'Stromspeicher [kWh]'] = battery_capacity + production_rest
                        battery_capacity += production_rest
                    else:
                        data.at[index, 'Stromspeicher [kWh]'] = batt_capacity 
                        battery_capacity = batt_capacity                
                else:
                    batt_consumption = (consumption-production) if (consumption-production) <= batt_15mins_peak else batt_15mins_peak
                    if batt_consumption <= battery_capacity:
                        data.at[index, 'Stromspeicher [kWh]'] = battery_capacity - batt_consumption
                        battery_capacity -= batt_consumption
                    else:
                        data.at[index, 'Stromspeicher [kWh]'] = 0.0
                        battery_capacity = 0.0    
                        
        return True
    except Exception as e:
        print(f"Fehler bei der Simulation der Batterienutzung: {e}")
        return False
    
def calculate_monthly_totals(user_type):
    global data
    
    data['Datum'] = pd.to_datetime(data['Datum'])
    
    monthly_consumption = data.groupby(data['Datum'].dt.month)['Stromverbrauch [kWh]'].sum()
 
    if user_type != 'consumer':
        monthly_production = data.groupby(data['Datum'].dt.month)['Stromerzeugung [kWh]'].sum()
    else:
        monthly_production = None
    return monthly_consumption, monthly_production

def create_graph_1(user_type):
    global data  
    if user_type == 'consumer':
        value_column = 'Stromverbrauch [kWh]'
        title_text = 'Lastprofil für ein Jahr im Viertelstundentakt'
        color_map = {value_column: 'blue'}
        df_graph_1 = data[['Datum', value_column]].copy()
        df_graph_1.columns = ['timestamp', value_column]
        
    if user_type == 'prosumer':
        value_column = ['Stromverbrauch [kWh]', 'Stromerzeugung [kWh]']
        title_text = 'Lastprofil für ein Jahr im Viertelstundentakt'
        color_map = {col: 'blue' if col == 'Stromverbrauch [kWh]' else 'green' for col in value_column}
        df_graph_1 = data[['Datum', 'Stromverbrauch [kWh]','Stromerzeugung [kWh]']].copy()
        df_graph_1.columns = ['timestamp', 'Stromverbrauch [kWh]', 'Stromerzeugung [kWh]']
    
    if user_type == 'flexumer':
        value_column = ['Stromverbrauch [kWh]', 'Stromerzeugung [kWh]', 'Stromspeicher [kWh]']  # Agregar 'Stromspeicher [kWh]'
        title_text = 'Lastprofil für ein Jahr im Viertelstundentakt'
        color_map = {col: 'blue' if col == 'Stromverbrauch [kWh]' else 'green' if col == 'Stromerzeugung [kWh]' else 'red' for col in value_column}  # Cambiar color_map
        df_graph_1 = data[['Datum', 'Stromverbrauch [kWh]', 'Stromerzeugung [kWh]', 'Stromspeicher [kWh]']].copy()  # Incluir 'Stromspeicher [kWh]'
        df_graph_1.columns = ['timestamp', 'Stromverbrauch [kWh]', 'Stromerzeugung [kWh]', 'Stromspeicher [kWh]']  # Cambiar nombres de columnas

    df_graph_1['timestamp'] = pd.to_datetime(df_graph_1['timestamp'])

    fig_1 = px.line(df_graph_1,
                    x='timestamp',
                    y=value_column,
                    title=title_text,
                    labels={'timestamp': 'Zeitpunkt', 'value': 'Strom [kWh]'},
                    color_discrete_map=color_map)
    
    fig_1.update_layout(legend=dict(title=""))

    fig_1.add_trace(go.Scatter(x=[df_graph_1['timestamp'].iloc[0], df_graph_1['timestamp'].iloc[-1]],
                               y=[df_graph_1['Stromverbrauch [kWh]'].mean(), df_graph_1['Stromverbrauch [kWh]'].mean()],
                               mode='lines',
                               name=f"AVG(Stromverbrauch): {df_graph_1['Stromverbrauch [kWh]'].mean():.2f} kWh/Viertelstunde",
                               line=dict(color='black', dash='dash'),
                               visible='legendonly'))

    if user_type != 'consumer' and 'Stromerzeugung [kWh]' in value_column:
        fig_1.add_trace(go.Scatter(x=[df_graph_1['timestamp'].iloc[0], df_graph_1['timestamp'].iloc[-1]],
                                   y=[df_graph_1['Stromerzeugung [kWh]'].mean(), df_graph_1['Stromerzeugung [kWh]'].mean()],
                                   mode='lines',
                                   name=f"AVG(Stromerzeugung): {df_graph_1['Stromerzeugung [kWh]'].mean():.2f} kWh/Viertelstunde",
                                   line=dict(color='black', dash='dash'),
                                   visible='legendonly'))
    
    if user_type == 'flexumer' and 'Stromspeicher [kWh]' in value_column:  # Agregar la línea para 'Stromspeicher [kWh]'
        fig_1.add_trace(go.Scatter(x=[df_graph_1['timestamp'].iloc[0], df_graph_1['timestamp'].iloc[-1]],
                                   y=[df_graph_1['Stromspeicher [kWh]'].mean(), df_graph_1['Stromspeicher [kWh]'].mean()],
                                   mode='lines',
                                   name=f"AVG(Stromspeicher): {df_graph_1['Stromspeicher [kWh]'].mean():.2f} kWh/Viertelstunde",
                                   line=dict(color='black', dash='dash'),
                                   visible='legendonly'))

    return fig_1
    

def create_graph_1_(user_type):
    global data  
    if user_type == 'consumer':
        value_column = 'Stromverbrauch [kWh]'
        title_text = 'Lastprofil für ein Jahr im Viertelstundentakt'
        color_map = {value_column: 'blue'}
        df_graph_1 = data[['Datum', value_column]].copy()
        df_graph_1.columns = ['timestamp', value_column]
    else:
        value_column = ['Stromverbrauch [kWh]', 'Stromerzeugung [kWh]']
        title_text = 'Lastprofil für ein Jahr im Viertelstundentakt'
        color_map = {col: 'blue' if col == 'Stromverbrauch [kWh]' else 'green' for col in value_column}
        df_graph_1 = data[['Datum', 'Stromverbrauch [kWh]','Stromerzeugung [kWh]']].copy()
        df_graph_1.columns = ['timestamp', 'Stromverbrauch [kWh]', 'Stromerzeugung [kWh]']

    df_graph_1['timestamp'] = pd.to_datetime(df_graph_1['timestamp'])

    fig_1 = px.line(df_graph_1,
                    x='timestamp',
                    y=value_column,
                    title=title_text,
                    labels={'timestamp': 'Zeitpunkt', 'value': 'Strom [kWh]'},
                    color_discrete_map=color_map)
    
    fig_1.update_layout(legend=dict(title=""))

    fig_1.add_trace(go.Scatter(x=[df_graph_1['timestamp'].iloc[0], df_graph_1['timestamp'].iloc[-1]],
                               y=[df_graph_1['Stromverbrauch [kWh]'].mean(), df_graph_1['Stromverbrauch [kWh]'].mean()],
                               mode='lines',
                               name=f"AVG(Stromverbrauch): {df_graph_1['Stromverbrauch [kWh]'].mean():.2f} kWh/Viertelstunde",
                               line=dict(color='black', dash='dash'),
                               visible='legendonly'))

    if user_type != 'consumer' and 'Stromerzeugung [kWh]' in value_column:
        fig_1.add_trace(go.Scatter(x=[df_graph_1['timestamp'].iloc[0], df_graph_1['timestamp'].iloc[-1]],
                                   y=[df_graph_1['Stromerzeugung [kWh]'].mean(), df_graph_1['Stromerzeugung [kWh]'].mean()],
                                   mode='lines',
                                   name=f"AVG(Stromerzeugung): {df_graph_1['Stromerzeugung [kWh]'].mean():.2f} kWh/Viertelstunde",
                                   line=dict(color='black', dash='dash'),
                                   visible='legendonly'))
    return fig_1

def create_graph_2(user_type):
    global data    
    monthly_consumption, monthly_production = calculate_monthly_totals(user_type)
    
    german_month_names = [
        'Januar', 'Februar', 'März', 'April', 'Mai', 'Juni',
        'Juli', 'August', 'September', 'Oktober', 'November', 'Dezember'
    ]

    fig_2 = go.Figure()

    if user_type == 'consumer':
        fig_2.add_trace(go.Bar(x=[german_month_names[i-1] for i in range(1, 13)],
                               y=monthly_consumption.values,
                               text=monthly_consumption.values.round(2), 
                               name='Stromverbrauch [kWh]', 
                               marker_color='blue', 
                               textposition='auto', 
                               textangle=0))
        
    elif user_type in ['prosumer', 'flexumer']:
        fig_2.add_trace(go.Bar(x=[german_month_names[i-1] for i in range(1, 13)], 
                               y=monthly_consumption.values, 
                               text=monthly_consumption.values.round(2), 
                               name='Stromverbrauch [kWh]', 
                               marker_color='blue', 
                               textposition='auto', 
                               textangle=0))

        if monthly_production is not None:
            fig_2.add_trace(go.Bar(x=[german_month_names[i-1] for i in range(1, 13)], 
                                   y=monthly_production.values, 
                                   text=monthly_production.values.round(2), 
                                   name='Stromerzeugung [kWh]', 
                                   marker_color='green',
                                   textposition='auto', 
                                   textangle=0))
        
        annual_consumption_mean = monthly_consumption.mean()
        annual_production_mean = monthly_production.mean() if monthly_production is not None else None

        fig_2.add_trace(go.Scatter(x=[german_month_names[i-1] for i in range(1, 13)], 
                                   y=[annual_consumption_mean] * 12,
                                   mode='lines', 
                                   name=f'AVG(Stromverbrauch): {annual_consumption_mean:.2f} kWh/Monat',
                                   line=dict(color='black', dash='dash', width=2),  visible='legendonly'))
        if annual_production_mean is not None:
            fig_2.add_trace(go.Scatter(x=[german_month_names[i-1] for i in range(1, 13)], 
                                       y=[annual_production_mean] * 12,
                                       mode='lines', 
                                       name=f'AVG(Stromerzeugung): {annual_production_mean:.2f} kWh/Monat',
                                       line=dict(color='black', dash='dash', width=2), visible='legendonly'))

    fig_2.update_layout(barmode='group', xaxis_title='Monat', yaxis_title='Strom [kWh]', title='Verteilung des Stromverbrauchs und der Stromerzeugung pro Monat')

    return fig_2


def create_graph_3():
    global data
    df_graph_3 = data[['Datum', 'Strommarktpreis [€/kWh]']].copy()
    df_graph_3.columns = ['timestamp', 'Tagespreis [€/kWh]']
    df_graph_3['timestamp'] = pd.to_datetime(df_graph_3['timestamp'])

    fig_3 = px.line(df_graph_3,
                    x='timestamp',
                    y=['Tagespreis [€/kWh]'],
                    title='Day-Ahead-Markt',
                    labels={'value': 'Strompreis [€/kWh]', 'timestamp': 'Zeitpunkt'},
                    color_discrete_map={'Tagespreis [€/kWh]': 'red'})
    
    fig_3.update_layout(legend=dict(title=""))
    
    fig_3.add_trace(go.Scatter(x=[df_graph_3['timestamp'].iloc[0], df_graph_3['timestamp'].iloc[-1]],
                               y=[data['Strommarktpreis [€/kWh]'].mean(), data['Strommarktpreis [€/kWh]'].mean()],
                               mode='lines',
                               name=f"AVG(Tagespreis): {data['Strommarktpreis [€/kWh]'].mean():.2f} €/kWh",
                               line=dict(color='black', dash='dash'),
                               visible='legendonly'))
    
    return fig_3

def create_graph_4():
    global data
    df_graph_4 = data[['Datum', 'Dynamischer Stromtarif [€/kWh]', 'Fixer Stromtarif [€/kWh]']].copy()
    df_graph_4.columns = ['timestamp', 'Dynamischer Stromtarif [€/kWh]', 'Fixer Stromtarif [€/kWh]']
    df_graph_4['timestamp'] = pd.to_datetime(df_graph_4['timestamp'])

    fig_4 = px.line(df_graph_4,
                    x='timestamp',
                    y=['Dynamischer Stromtarif [€/kWh]', 'Fixer Stromtarif [€/kWh]'],
                    title='Strompreisentwicklung',
                    labels={'value': 'Strompreis [€/kWh]', 'timestamp': 'Zeitpunkt'},
                    color_discrete_map={'Dynamischer Stromtarif [€/kWh]': 'olive', 'Fixer Stromtarif [€/kWh]': 'orange'})
    
    fig_4.update_layout(legend=dict(title=""))
            
    fig_4.add_trace(go.Scatter(x=[df_graph_4['timestamp'].iloc[0], df_graph_4['timestamp'].iloc[-1]],
                               y=[data['Dynamischer Stromtarif [€/kWh]'].mean(), data['Dynamischer Stromtarif [€/kWh]'].mean()],
                               mode='lines',
                               name=f"AVG(Dynamischer Stromtarif): {data['Dynamischer Stromtarif [€/kWh]'].mean():.2f} €/Viertelstunde",
                               line=dict(color='black', dash='dash'),
                               visible='legendonly'))

    fig_4.add_trace(go.Scatter(x=[df_graph_4['timestamp'].iloc[0], df_graph_4['timestamp'].iloc[-1]],
                               y=[data['Fixer Stromtarif [€/kWh]'].mean(), data['Fixer Stromtarif [€/kWh]'].mean()],
                               mode='lines',
                               name=f"AVG(Fixer Stromtarif): {data['Fixer Stromtarif [€/kWh]'].mean():.2f} €/Viertelstunde",
                               line=dict(color='black', dash='dash'),
                               visible='legendonly'))  
    return fig_4

def create_graph_5():
    global data
    monthly_fixed_tariff = data.groupby(df['Datum'].dt.month)['Fixer Stromtarif [€/kWh]'].sum()
    monthly_variable_tariff = data.groupby(df['Datum'].dt.month)['Dynamischer Stromtarif [€/kWh]'].sum()
    
    german_month_names = [
        'Januar', 'Februar', 'März', 'April', 'Mai', 'Juni',
        'Juli', 'August', 'September', 'Oktober', 'November', 'Dezember'
    ]

    fig_5 = go.Figure()

    fig_5.add_trace(go.Bar(x=[calendar.month_name[i] for i in range(1, 13)],
                           y=monthly_fixed_tariff.values,
                           name='Fixer Stromtarif [€/Monat]',
                           marker_color='orange',
                           text=monthly_fixed_tariff.round(2),
                           textposition='auto',
                           textangle=0))
    
    fig_5.add_trace(go.Bar(x=[calendar.month_name[i] for i in range(1, 13)],
                           y=monthly_variable_tariff.values,
                           name='Dynamischer Stromtarif [€/Monat]',
                           marker_color='olive',
                           text=monthly_variable_tariff.round(2),
                           textposition='auto', textangle=0))

    fig_5.update_layout(barmode='group', xaxis_title='Monat', yaxis_title='Tarifpreis in €', title='Stromtarife [€/Monat]')

    annual_fixed_tariff_mean = monthly_fixed_tariff.mean()
    annual_variable_tariff_mean = monthly_variable_tariff.mean()

    fig_5.add_trace(go.Scatter(x=[calendar.month_name[i] for i in range(1, 13)],
                               y=[annual_fixed_tariff_mean] * 12,
                               mode='lines', name=f'AVG(Fixer Stromtarif): {annual_fixed_tariff_mean:.2f} €/Monat',
                               line=dict(color='black', dash='dash', width=2), visible='legendonly'))
    
    fig_5.add_trace(go.Scatter(x=[calendar.month_name[i] for i in range(1, 13)],
                               y=[annual_variable_tariff_mean] * 12,
                               mode='lines', name=f'AVG(Dynamischer Stromtarif): {annual_variable_tariff_mean:.2f} €/Monat',
                               line=dict(color='black', dash='dash', width=2), visible='legendonly'))

    return fig_5

def save_to_excel():
    global data
    try:
        excel_file_path = 'Output-Data/Simulationsergebnis.xlsx'
        data.to_excel(excel_file_path, index=False)
    except Exception as e:
        print(f"Fehler beim Speichern von DataFrame in Excel: {e}")
    
##############################################################################################################
# Callbacks
##############################################################################################################
@app.callback(
    Output('dynamic-image', 'src'),
    [Input('user-type-radio', 'value')]
)
def update_image(user_type):
    return load_image(user_type)

@app.callback(
    Output('pv-capacity', 'options'),
    [Input('user-type-radio', 'value')]
)
def update_pv_capacity_options(user_type):
    if user_type == 'consumer':
        return [{'label': 'N/A', 'value': 'N/A'}]
    else:
        return [
            {'label': '1 kWp', 'value': 'PV-1kWp'},
            {'label': '5 kWp', 'value': 'PV-5kWp'},
            {'label': '10 kWp', 'value': 'PV-10kWp'},
            {'label': '15 kWp', 'value': 'PV-15kWp'},
            {'label': '20 kWp', 'value': 'PV-20kWp'},
            {'label': '25 kWp', 'value': 'PV-25kWp'},
            {'label': '30 kWp', 'value': 'PV-30kWp'}
        ]
    
@app.callback(
    Output('pv-capacity', 'value'),
    [Input('user-type-radio', 'value')]
)
def update_pv_capacity_value(user_type):
    if user_type == 'consumer':
        return 'N/A'
    else:
        return 'PV-1kWp'

@app.callback(
    Output('battery-capacity', 'options'),
    [Input('user-type-radio', 'value')]
)
def update_battery_capacity_options(user_type):
    if user_type != 'flexumer':
        return [{'label': 'N/A', 'value': 'N/A'}]
    else:    
        return [
            {'label': '2,5 kWh', 'value': '2,5 kWh'},
            {'label': '5,1 kWh', 'value': '5,1 kWh'},
            {'label': '7,7 kWh', 'value': '7,7 kWh'},
            {'label': '10,2 kWh', 'value': '10,2 kWh'},
            {'label': '12,8 kWh', 'value': '12,8 kWh'},
        ]
@app.callback(
    Output('battery-capacity', 'value'),
    [Input('user-type-radio', 'value')]
)
def update_battery_capacity_value(user_type):
    if user_type != 'flexumer':
        return 'N/A'
    else:
        return '2,5 kWh'

@app.callback(
    [Output('simulation-view', 'style'),
     Output('simulation-results', 'children')],
    [Input('simulate-button', 'n_clicks')],
    [State('annual-consumption-input', 'value'),
     State('current-tariff-input', 'value'),
     State('current-base-input', 'value'),
     State('pv-capacity', 'value'),
     State('user-type-radio', 'value'),
     State('battery-capacity', 'value')]
)
def update_simulation_results(n_clicks, annual_input, tariff_input, base_input, pv_capacity, user_type, battery_capacity):
    global data

    if n_clicks > 0 and annual_input is not None and tariff_input is not None and base_input is not None:
        simulation_successful_1 = simulate_power_consumption(annual_input, tariff_input)
        simulation_successful_2 = simulate_power_production(pv_capacity)
        simulation_successful_3 = simulate_power_price(annual_input, tariff_input, base_input)
        simulation_successful_4 = simulate_battery_usage(battery_capacity)

        if simulation_successful_1 and simulation_successful_2 and simulation_successful_2 is not None:
            fig_1 = create_graph_1(user_type)
            fig_2 = create_graph_2(user_type)
            fig_3 = create_graph_3()
            fig_4 = create_graph_4()
            fig_5 = create_graph_5()

            return {'display': 'block'}, [
                html.Hr(),
                dcc.Graph(id='graph-1', figure=fig_1),
                html.Hr(),
                dcc.Graph(id='graph-2', figure=fig_2),
                html.Hr(),
                dcc.Graph(id='graph-3', figure=fig_3),
                html.Hr(),
                dcc.Graph(id='graph-4', figure=fig_4),
                html.Hr(),
                dcc.Graph(id='graph-5', figure=fig_5),
            ]
        else:
            return {'display': 'none'}, ''
    else:
        return {'display': 'none'}, ''

@app.callback(
    Output('save-button', 'n_clicks'),
    [Input('save-button', 'n_clicks')]
)
def save_to_excel_callback(n_clicks):
    if n_clicks is not None and n_clicks > 0:
        save_to_excel()
    return 0
    
##############################################################################################################
# Start
##############################################################################################################
load_data()
app.layout = build_layout()

Error: No se pudo encontrar el archivo Output-Data/ETL-Prozess_Ergebnisse.xlsx


In [2]:
def run_simulations_app():
    app.run_server(debug=True, port=8050)